Double Play Analytics

In [1]:
#Import Packages
import pandas as pd

#### Create a single dataset

In [2]:
file_paths = ["data/gl2010.txt", "data/gl2011.txt", "data/gl2012.txt", "data/gl2013.txt", "data/gl2014.txt",
              "data/gl2015.txt", "data/gl2016.txt", "data/gl2017.txt", "data/gl2018.txt", "data/gl2019.txt",
              "data/gl2020.txt", "data/gl2021.txt", "data/gl2022.txt", "data/gl2023.txt"]

In [3]:
dfs=[]

In [4]:
for file_path in file_paths:
    with open(file_path, "r") as f:
        # Read lines from the file
        data = f.readlines()
        # Split each line by comma and create a list of lists
        data_split = [line.strip().split(",") for line in data]
        # Create DataFrame from the data
        df_initial = pd.DataFrame(data_split)
        # Append DataFrame to the list
        dfs.append(df_initial)

In [5]:
# Concatenate all DataFrames in the list
df = pd.concat(dfs, ignore_index=True)

#### Understand what the data look like

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
0,"""20100404""","""0""","""Sun""","""NYA""","""AL""",1,"""BOS""","""AL""",1,7,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,"""20100405""","""0""","""Mon""","""MIN""","""AL""",1,"""ANA""","""AL""",1,3,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,"""20100405""","""0""","""Mon""","""CLE""","""AL""",1,"""CHA""","""AL""",1,0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,"""20100405""","""0""","""Mon""","""DET""","""AL""",1,"""KCA""","""AL""",1,8,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,"""20100405""","""0""","""Mon""","""SEA""","""AL""",1,"""OAK""","""AL""",1,5,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
32479,"""20231001""","""0""","""Sun""","""SDN""","""NL""",162,"""CHA""","""AL""",162,2,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32480,"""20231001""","""0""","""Sun""","""CLE""","""AL""",162,"""DET""","""AL""",162,2,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32481,"""20231001""","""0""","""Sun""","""NYA""","""AL""",162,"""KCA""","""AL""",162,2,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32482,"""20231001""","""0""","""Sun""","""TEX""","""AL""",162,"""SEA""","""AL""",162,0,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32483,"""20231001""","""0""","""Sun""","""TBA""","""AL""",162,"""TOR""","""AL""",162,12,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
nan_values = df.isna().any()
nan_values

In [ ]:
df.dtypes

In [ ]:
df.count()

#### Preparing data for more EDA (easier to use with names)

In [8]:
new_column_names = {
    0: 'date',
    1: 'num_games',
    2: 'day_of_week',
    3: 'team_visiting',
    4: 'league_visiting',
    5: 'game_num_visiting',
    6: 'team_home',
    7: 'league_home',
    8: 'game_num_home',
    9: 'score_visiting',
    10: 'score_home',
    11: 'outs_in_game',
    12: 'time_of_day',
    13: 'game_completed',
    14: 'forfeit',
    15: 'protest',
    16: 'park_id',
    17: 'attendance',
    18: 'length_min',
    19: 'line_score_visiting',
    20: 'line_score_home',    
    21: 'at_bats_visiting',
    22: 'hits_visiting',
    23: 'double_visiting',
    24: 'triple_visiting',
    25: 'home_run_visiting',
    26: 'rbi_visiting',
    27: 'sacrifice_hit_visiting',
    28: 'sacrifine_fly_visiting',
    29: 'hit_by_pitch_visiting',
    30: 'walk_visiting',
    31: 'intent_walk_visiting',
    32: 'strikeout_visiting',
    33: 'stolen_base_visiting',
    34: 'caught_stealing_visiting',
    35: 'grounded_into_double_plays_visiting',
    36: 'first_catcher_interfere_visiting',
    37: 'left_on_base_visiting',
    38: 'pitchers_used_visiting',
    39: 'individual_earned_runs_visiting',
    40: 'team_earned_runs_visiting',
    41: 'wild_pitches_visiting',
    42: 'balks_visiting',
    43: 'putouts_visiting',   
    44: 'assists_visiting',
    45: 'errors_visiting',
    46: 'passed_balls_visiting',
    47: 'double_def_visiting',
    48: 'triple_def_visiting',       
    49: 'at_bats_home',
    50: 'hits_home',
    51: 'double_home',
    52: 'triple_home',
    53: 'home_run_home',
    54: 'rbi_home',
    55: 'sacrifice_hit_home',
    56: 'sacrifine_fly_home',
    57: 'hit_by_pitch_home',
    58: 'walk_home',
    59: 'intent_walk_home',
    60: 'strikeout_home',
    61: 'stolen_base_home',
    62: 'caught_stealing_home',
    63: 'grounded_into_double_plays_home',
    64: 'first_catcher_interfere_home',
    65: 'left_on_base_home',
    66: 'pitchers_used_home',
    67: 'individual_earned_runs_home',
    68: 'team_earned_runs_home',
    69: 'wild_pitches_home',
    70: 'balks_home',
    71: 'putouts_home',
    72: 'assists_home',
    73: 'errors_home',
    74: 'passed_balls_home',
    75: 'double_def_home',
    76: 'triple_def_home',    
    77: 'home_ump_id',
    78: 'home_ump_name',
    79: '1B_ump_id',
    80: '1B_ump_name',
    81: '2B_ump_id',
    82: '2B_ump_name',
    83: '3B_ump_id',
    84: '3B_ump_name',
    85: 'LF_ump_id',
    86: 'LF_ump_name',
    87: 'RF_ump_id',
    88: 'RF_ump_name',
    89: 'manager_id_visiting',
    90: 'manager_name_visiting',
    91: 'manager_id_home',
    92: 'manager_name_home',
    93: 'winning_pitcher_id',
    94: 'winning_pitcher_name',
    95: 'losing_pitcher_id',
    96: 'losing_pitcher_name',
    97: 'saving_pitcher_id',
    98: 'saving_pitcher_name',
    99: 'game_winning_rbi_id',
    100: 'game_winning_rbi_name',
    101: 'starting_pitcher_id_visiting',
    102: 'starting_pitcher_name_visiting',
    103: 'starting_pitcher_id_home',
    104: 'starting_pitcher_name_home',
    105: 'player1_id_visiting',
    106: 'player1_name_visiting',
    107: 'player1_position_visiting',
    108: 'player2_id_visiting',
    109: 'player2_name_visiting',
    110: 'player2_position_visiting',
    111: 'player3_id_visiting',
    112: 'player3_name_visiting',
    113: 'player3_position_visiting',
    114: 'player4_id_visiting',
    115: 'player4_name_visiting',
    116: 'player4_position_visiting',
    117: 'player5_id_visiting',
    118: 'player5_name_visiting',
    119: 'player5_position_visiting',
    120: 'player6_id_visiting',
    121: 'player6_name_visiting',
    122: 'player6_position_visiting',
    123: 'player7_id_visiting',
    124: 'player7_name_visiting',
    125: 'player7_position_visiting',
    126: 'player8_id_visiting',
    127: 'player8_name_visiting',
    128: 'player8_position_visiting',
    129: 'player9_id_visiting',
    130: 'player9_name_visiting',
    131: 'player9_position_visiting',    
    132: 'player1_id_home',
    133: 'player1_name_home',
    134: 'player1_position_home',
    135: 'player2_id_home',
    136: 'player2_name_home',
    137: 'player2_position_home',
    138: 'player3_id_home',
    139: 'player3_name_home',
    140: 'player3_position_home',
    141: 'player4_id_home',
    142: 'player4_name_home',
    143: 'player4_position_home',
    144: 'player5_id_home',
    145: 'player5_name_home',
    146: 'player5_position_home',
    147: 'player6_id_home',
    148: 'player6_name_home',
    149: 'player6_position_home',
    150: 'player7_id_home',
    151: 'player7_name_home',
    152: 'player7_position_home',
    153: 'player8_id_home',
    154: 'player8_name_home',
    155: 'player8_position_home',
    156: 'player9_id_home',
    157: 'player9_name_home',
    158: 'player9_position_home',
    159: 'misc',
    160: 'acquisition'
}

# Rename columns
df.rename(columns=new_column_names, inplace=True)

In [9]:
df.drop(columns=df.columns[161:179], inplace=True)


In [10]:
# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the head of the DataFrame
print(df.head())

         date num_games day_of_week team_visiting league_visiting  \
0  "20100404"       "0"       "Sun"         "NYA"            "AL"   
1  "20100405"       "0"       "Mon"         "MIN"            "AL"   
2  "20100405"       "0"       "Mon"         "CLE"            "AL"   
3  "20100405"       "0"       "Mon"         "DET"            "AL"   
4  "20100405"       "0"       "Mon"         "SEA"            "AL"   

  game_num_visiting team_home league_home game_num_home score_visiting  \
0                 1     "BOS"        "AL"             1              7   
1                 1     "ANA"        "AL"             1              3   
2                 1     "CHA"        "AL"             1              0   
3                 1     "KCA"        "AL"             1              8   
4                 1     "OAK"        "AL"             1              5   

  score_home outs_in_game time_of_day game_completed forfeit protest  park_id  \
0          9           51         "N"             ""      "

In [11]:
columns_to_convert = [
    "num_games", 
    "game_num_visiting", 
    "game_num_home", 
    "score_visiting", 
    'game_num_home', 
    'score_visiting',    
    'score_home', 
    "outs_in_game", 
    "attendance", 
    "length_min", 
    "at_bats_visiting",                      
    "hits_visiting", 
    "double_visiting", 
    "triple_visiting", 
    "home_run_visiting", 
    "rbi_visiting", 
    "sacrifice_hit_visiting",  
    'sacrifine_fly_visiting',
    'hit_by_pitch_visiting',
    'walk_visiting',
    'intent_walk_visiting',
    'strikeout_visiting',
    'stolen_base_visiting',
    'caught_stealing_visiting',
    'grounded_into_double_plays_visiting',
    'first_catcher_interfere_visiting',
    'left_on_base_visiting',
    'pitchers_used_visiting',
    'individual_earned_runs_visiting',
    'team_earned_runs_visiting',
    'wild_pitches_visiting',
    'balks_visiting',
    'putouts_visiting',   
    'assists_visiting',
    'errors_visiting',
    'passed_balls_visiting',
    'double_def_visiting',
    'triple_def_visiting',       
    'at_bats_home',
    'hits_home',
    'double_home',
    'triple_home',
    'home_run_home',
    'rbi_home',
    'sacrifice_hit_home',
    'sacrifine_fly_home',
    'hit_by_pitch_home',
    'walk_home',
    'intent_walk_home',
    'strikeout_home',
    'stolen_base_home',
    'caught_stealing_home',
    'grounded_into_double_plays_home',
    'first_catcher_interfere_home',
    'left_on_base_home',
    'pitchers_used_home',
    'individual_earned_runs_home',
    'team_earned_runs_home',
    'wild_pitches_home',
    'balks_home',
    'putouts_home',
    'assists_home',
    'errors_home',
    'passed_balls_home',
    'double_def_home',
    'triple_def_home']

# Player positions are numeric, but are being left as strings

In [12]:
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [13]:
df.head()

,date,num_games,day_of_week,team_visiting,league_visiting,game_num_visiting,team_home,league_home,game_num_home,score_visiting,score_home,outs_in_game,time_of_day,game_completed,forfeit,protest,park_id,attendance,length_min,line_score_visiting,line_score_home,at_bats_visiting,hits_visiting,double_visiting,triple_visiting,home_run_visiting,rbi_visiting,sacrifice_hit_visiting,sacrifine_fly_visiting,hit_by_pitch_visiting,walk_visiting,intent_walk_visiting,strikeout_visiting,stolen_base_visiting,caught_stealing_visiting,grounded_into_double_plays_visiting,first_catcher_interfere_visiting,left_on_base_visiting,pitchers_used_visiting,individual_earned_runs_visiting,team_earned_runs_visiting,wild_pitches_visiting,balks_visiting,putouts_visiting,assists_visiting,errors_visiting,passed_balls_visiting,double_def_visiting,triple_def_visiting,at_bats_home,hits_home,double_home,triple_home,home_run_home,rbi_home,sacrifice_hit_home,sacrifine_fly_home,hit_by_pitch_home,walk_home,intent_walk_home,strikeout_home,stolen_base_home,caught_stealing_home,grounded_into_double_plays_home,first_catcher_interfere_home,left_on_base_home,pitchers_used_home,individual_earned_runs_home,team_earned_runs_home,wild_pitches_home,balks_home,putouts_home,assists_home,errors_home,passed_balls_home,double_def_home,triple_def_home,home_ump_id,home_ump_name,1B_ump_id,1B_ump_name,2B_ump_id,2B_ump_name,3B_ump_id,3B_ump_name,LF_ump_id,LF_ump_name,RF_ump_id,RF_ump_name,manager_id_visiting,manager_name_visiting,manager_id_home,manager_name_home,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,game_winning_rbi_id,game_winning_rbi_name,starting_pitcher_id_visiting,starting_pitcher_name_visiting,starting_pitcher_id_home,starting_pitcher_name_home,player1_id_visiting,player1_name_visiting,player1_position_visiting,player2_id_visiting,player2_name_visiting,player2_position_visiting,player3_id_visiting,player3_name_visiting,player3_position_visiting,player4_id_visiting,player4_name_visiting,player4_position_visiting,player5_id_visiting,player5_name_visiting,player5_position_visiting,player6_id_visiting,player6_name_visiting,player6_position_visiting,player7_id_visiting,player7_name_visiting,player7_position_visiting,player8_id_visiting,player8_name_visiting,player8_position_visiting,player9_id_visiting,player9_name_visiting,player9_position_visiting,player1_id_home,player1_name_home,player1_position_home,player2_id_home,player2_name_home,player2_position_home,player3_id_home,player3_name_home,player3_position_home,player4_id_home,player4_name_home,player4_position_home,player5_id_home,player5_name_home,player5_position_home,player6_id_home,player6_name_home,player6_position_home,player7_id_home,player7_name_home,player7_position_home,player8_id_home,player8_name_home,player8_position_home,player9_id_home,player9_name_home,player9_position_home,misc,acquisition
0,"""20100404""",NaN,"""Sun""","""NYA""","""AL""",1,"""BOS""","""AL""",1,7,9,51,"""N""","""""","""""","""""","""BOS07""",37440.0,226.0,"""020300200""","""01001331x""",37.0,12,2.0,0.0,2,6,0,0,0,6,0,2,2,0,2,0,9,5,8,8,1,0,24,9,1,1,1,0,34,12,3,1,1,8,0,1,0,4,0,5,0,0,0,0,6,6,7,7,1,0,27,15,0,0,2,0,"""westj901""","""Joe West""","""herna901""","""Angel Hernandez""","""schrp901""","""Paul Schrieber""","""drakr901""","""Rob Drake""","""""","""(none)""","""""","""(none)""","""giraj001""","""Joe Girardi""","""frant001""","""Terry Francona""","""okajh001""","""Hideki Okajima""","""parkc002""","""Chan Ho Park""","""papej001""","""Jonathan Papelbon""","""""","""(none)""","""sabac001""","""CC Sabathia""","""beckj002""","""Josh Beckett""","""jeted001""","""Derek Jeter""",6,"""johnn001""","""Nick Johnson""",10,"""teixm001""","""Mark Teixeira""",3,"""rodra001""","""Alex Rodriguez""",5,"""canor001""","""Robinson Cano""",4,"""posaj001""","""Jorge Posada""",2,"""granc001""","""Curtis Granderson""",8,"""swisn001""","""Nick Swisher""",9,"""gardb001""","""Brett Gardner""",7,"""ell

In [20]:
df["game_num_visiting"].describe()

count    32484.000000
mean        80.078654
std         46.946529
min          1.000000
25%         39.000000
50%         79.000000
75%        121.000000
max        163.000000
Name: game_num_visiting, dtype: float64

In [ ]:
#################I want to see .info for all features, want to look at missing values-which features have missing. Below code not working yet
# Set the display width to accommodate more characters per row
pd.set_option('display.width', 1000)  # Adjust as needed

# Set the maximum number of rows and columns to None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display DataFrame information
print(df.info())

#### Crosstabs and Value Counts

In [ ]:
#cross_tab = pd.crosstab(df['team_home'], df['outs_in_game'])
#print(cross_tab)

In [16]:
df['strikeout_visiting'].value_counts()

8     4370
7     4197
9     4037
6     3618
10    3426
11    2683
5     2598
12    1872
4     1598
13    1143
3      851
14     708
15     424
2      350
16     247
17     120
1       99
18      47
0       41
19      31
20      13
24       5
22       2
21       1
23       1
25       1
26       1
Name: strikeout_visiting, dtype: int64

In [28]:
cross_tab2 = pd.crosstab(df['strikeout_visiting'], df['year'])
print(cross_tab2)

year                2010  2011  2012  2013  2014  2015  2016  2017  2018  \
strikeout_visiting                                                         
0                      5     3     2     4     5     4     2     1     1   
1                     17    20     6     3    12     7     5     5     5   
2                     47    55    34    33    37    23    33    15    16   
3                    117   107    89    84    80    64    54    45    41   
4                    188   205   137   157   115   135   123    91    90   
5                    257   256   228   211   228   240   176   184   168   
6                    345   327   320   328   284   322   257   266   222   
7                    323   338   339   330   336   318   323   328   306   
8                    305   322   359   316   338   337   343   350   307   
9                    288   269   287   309   301   283   304   287   334   
10                   214   217   219   218   219   242   261   263   288   
11          

#### Create new features

In [ ]:
# Creating a feature for the winning team for each game/row
def compare_and_get_value(row):
    if row['score_visiting'] > row['score_home']:
        return row['team_visiting']
    else:
        return row['team_home']

# Create a new feature based on the comparison
df['winning_team'] = df.apply(compare_and_get_value, axis=1)

#################check to see if there was ever a tie or null value in this category

In [26]:
# Creating an integer four-digit year feature

# Extract the first four characters
year_digits = df['date'].str[1:5]

# Convert the extracted substring to numeric
df['year'] = pd.to_numeric(year_digits)

# Print the result
print(df["year"])


0        2010
1        2010
2        2010
3        2010
4        2010
5        2010
6        2010
7        2010
8        2010
9        2010
10       2010
11       2010
12       2010
13       2010
14       2010
15       2010
16       2010
17       2010
18       2010
19       2010
20       2010
21       2010
22       2010
23       2010
24       2010
25       2010
26       2010
27       2010
28       2010
29       2010
30       2010
31       2010
32       2010
33       2010
34       2010
35       2010
36       2010
37       2010
38       2010
39       2010
40       2010
41       2010
42       2010
43       2010
44       2010
45       2010
46       2010
47       2010
48       2010
49       2010
50       2010
51       2010
52       2010
53       2010
54       2010
55       2010
56       2010
57       2010
58       2010
59       2010
60       2010
61       2010
62       2010
63       2010
64       2010
65       2010
66       2010
67       2010
68       2010
69       2010
70       2010
71    

In [27]:
df['year'] = df['year'].astype(int)

In [ ]:
df['winning_team'].value_counts()

In [ ]:
df["day_of_week"].value_counts()

#### Notes - Working

Data Notes: 
- COVID years
    - 2020 less games

Other feature engineering
   - run differential 
   - year
   
Update data type (year, integer/float)